In [1]:
!pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 1.4 MB/s 


In [9]:
import os
import sys
import numpy as np
import rasterio
import pandas as pd
import rasterio.features
import matplotlib.pyplot as plt
csv_path = "./drive/MyDrive/Colab Notebooks/Thesis_Deliverable/Resources/Midkin - Midkin - 04 - 2018 - Harvest - Cotton [d5c28bb3-6c38-4dd6-a081-c73713751779].csv"

df = pd.read_csv(csv_path)
print(min(df["Longitude"]))
print(max(df["Longitude"]))
print(min(df["Latitude"]))
print(max(df["Latitude"]))



149.769735192548
149.77977637345
-29.3438397245842
-29.3307674445558


In [11]:
import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=YZq6WfJqospZJ8L0yCYoe4QOAjAABwkTwte0lHiEOoI&tc=ZqLzIKTORMUKq9x8ccfFfBCUWlOBGZRZKskH1KcoxaI&cc=7kRqG7fORrcbqIBWQL81p4NCZK88isi4vfE4ZDUgL0k

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfgeXvuiO3WQwwZ62GN_nI1h-pVOAZaOATEH6pZBymQSn2szgCI0DcyYj4E

Successfully saved authorization token.


In [13]:
ee.Initialize()

In [14]:
# This is the cloud masking function provided by GEE but adapted for use in Python.
def maskS2clouds(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)
    mask = mask.bitwiseAnd(cirrusBitMask).eq(0)
    return image.updateMask(mask).divide(10000)
# Define the geometry of the area for which you would like images.
geom = ee.Geometry.Polygon([[149.77977637345, -29.3438397245842],
                            [149.769735192548, -29.3307674445558],
                            [149.77977637345, -29.3438397245842],
                            [149.77977637345, -29.3307674445558]])
geom1 = ee.Geometry.Polygon([[149.769735192548,-29.340444368439442],
          [149.77561476256292,-29.34330621763878],
          [149.7759366276447,-29.343119171125874],
          [149.7774815800373,-29.3438397245842],
          [149.77977637345,-29.339995443627142],
          [149.77585079695623,-29.3307674445558],
          [149.769735192548,-29.340444368439442]])
def crop(img):
  return img.clip(geom1)

In [15]:
dateStart = "2017-10-01"
dateEnd = "2018-05-16"
dates = pd.date_range(start=dateStart, freq="20D" ,end=dateEnd)
len(dates)

12

In [16]:


for i in range(len(dates)-1):
# Call collection of satellite images.
  collection = (ee.ImageCollection("COPERNICUS/S2")
                # Select the Red, Green and Blue image bands, as well as the cloud masking layer.
                .select(['QA60','B2', 'B3', 'B4','B8','B5','B6','B7','B8A','B11','B12'])
                # Filter for images within a given date range.
                .filter(ee.Filter.date( str(dates[i].date()), str(dates[i+1].date()) ))
                # Filter for images that overlap with the assigned geometry.
                .filterBounds(geom)
                # Filter for images that have less then 20% cloud coverage.
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
                # Apply cloud mask.
                .map(maskS2clouds)).map(crop)
  # Sort images in the collection by index (which is equivalent to sorting by date), 
  # with the oldest images at the front of the collection.
  # Convert collection into a single image mosaic where only images at the top of the collection are visible.
  image = collection.sort('system:index', opt_ascending=False).mosaic()
  # Assign visualization parameters to the image.
  # image = image.visualize(bands=['B4', 'B3', 'B2'],
  #                         min=[0.0, 0.0, 0.0],
  #                         max=[0.3, 0.3, 0.3])
  # Assign export parameters.
  task_config = {
      'region': geom.coordinates().getInfo(),
      'folder': './drive/MyDrive/Colab Notebooks/Thesis_Deliverable/Resources/Sentinel-2',
      'scale': 10,
      'crs': 'EPSG:4326',
      'description': 'Date_'+str(dates[i+1].date())
  }
  # Export Image
  task = ee.batch.Export.image.toDrive(image, **task_config)
  task.start()
  # break



In [19]:
ee.batch.Task.list()[:]

[<Task M6DLRCKKYUUMJHBTLTVLTOW3 EXPORT_IMAGE: Date_2018-05-09 (COMPLETED)>,
 <Task 2SD7N63QSTCCZHG2C3XZ5UIW EXPORT_IMAGE: Date_2018-04-19 (COMPLETED)>,
 <Task 3HV5MQBKM7RDHUCJAKT4W4CV EXPORT_IMAGE: Date_2018-03-30 (COMPLETED)>,
 <Task LW3ZK6PZDDUBP47SHT75CS7U EXPORT_IMAGE: Date_2018-03-10 (COMPLETED)>,
 <Task 5BQDKVGQ2QSBZAEP2CBJRFTT EXPORT_IMAGE: Date_2018-02-18 (COMPLETED)>,
 <Task 6JBSMKCTQSZZEI7KW3JOBUK4 EXPORT_IMAGE: Date_2018-01-29 (COMPLETED)>,
 <Task ZHB3SDZ3PZLUSORXR5A76MHO EXPORT_IMAGE: Date_2018-01-09 (COMPLETED)>,
 <Task LZML3AEPLYGIXC6NGMFSZ5SW EXPORT_IMAGE: Date_2017-12-20 (COMPLETED)>,
 <Task HVUJJ2ZHKU5MKSUCTTIHWW3A EXPORT_IMAGE: Date_2017-11-30 (FAILED)>,
 <Task QAEDKDQWUD7QYKMKGPOCG4QD EXPORT_IMAGE: Date_2017-11-10 (COMPLETED)>,
 <Task OUHUAPGUOIVATXPYUNKRWENC EXPORT_IMAGE: Date_2017-10-21 (COMPLETED)>]